<a href="https://colab.research.google.com/github/Kushal190903/unreliable-news-detection/blob/main/unreliable_news_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install langchain_huggingface
!pip install langchain


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.2/292.2 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.2 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import re
import string
import numpy as np
# from time import perf_counter
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
# from tensorflow.keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense,Dropout
from keras.regularizers import l2
from langchain_huggingface import HuggingFaceEmbeddings
from sklearn.metrics import classification_report,confusion_matrix
import xgboost as xgb

In [ ]:

def wordopt(t):
    t = t.lower()
    t = re.sub('\[.*?\]', '', t)
    t = re.sub("\\W"," ",t)
    t = re.sub('https?://\S+|www\.\S+', '', t)
    t = re.sub('<.*?>+', '', t)
    t = re.sub('[%s]' % re.escape(string.punctuation), '', t)
    t = re.sub('\n', '', t)
    t = re.sub('\w*\d\w*', '', t)
    return t

In [ ]:
dataframe1 = pd.read_csv("/content/drive/MyDrive/fake news detection files/train.csv")
dataframe1=dataframe1[['text','label']]
dataframe2_fake = pd.read_csv("/content/drive/MyDrive/fake news detection files/faken.csv")
dataframe2_true = pd.read_csv("/content/drive/MyDrive/fake news detection files/true.csv")
dataframe2_true["label"] = 0
dataframe2_fake["label"] = 1
# print(dataframe2_true.head(3))
# print(dataframe2_fake.head(3))
dataframe2_merge = pd.concat([dataframe2_fake, dataframe2_true], axis=0)

dataframe2 = dataframe2_merge.drop(["title", "subject","date"], axis=1)

# dataframe2.drop(["index"], axis=1, inplace=True)
dataframe=pd.concat([dataframe1,dataframe2],axis=0)
dataframe=dataframe.sample(frac=1)
dataframe.reset_index(inplace=True)
dataframe.drop(["index"], axis=1, inplace=True)
dataframe

,text,label
0,Shocking surveillance video of two police offi...,0
1,What in the world is she thinking? She says th...,1
2,Donald Trump s nominee for a federal judgeship...,1
3,Donald J. Trump’s has discussed the possibi...,0
4,HANOI (Reuters) - U.S. President Donald Trump ...,0
...,...,...
65714,If you ve ever watched Donald Trump and his wi...,1
65715,"All around the world, millions of smartphone u...",0
65716,While we (the U.S. gov't) have been spending m...,1
65717,The domestic terrorists who are holed up in th...,1


In [ ]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65719 entries, 0 to 65718
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    65680 non-null  object
 1   label   65719 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.0+ MB


In [ ]:
dataframe=dataframe.dropna()


In [ ]:
x=dataframe['text']
y=dataframe['label']
# x=x.apply(wordopt)
# tokenizer=Tokenizer()
# tokenizer.fit_on_texts(x)
# x=tokenizer.texts_to_sequences(x)
# x=pad_sequences(x,maxlen=1000,padding='post',truncating='post')

embedding=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
x=embedding.embed_documents(x)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
vec=embedding.embed_query("hello i am kushal")
len(vec)

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,test_size=0.2,random_state=42)


In [ ]:
len(x_train[0])

In [ ]:
model=Sequential()
model.add(Dense(64,activation='relu',input_dim=384))
model.add(Dense(64,activation='relu'))



model.add(Dense(32,activation='relu',kernel_regularizer=l2(0.01)))
model.add(Dropout(0.2))

model.add(Dense(1,activation='sigmoid'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.fit(np.array(x_train),np.array(y_train),epochs=10,batch_size=32,validation_data=(np.array(x_val),np.array(y_val)))

Epoch 1/10
1314/1314 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.7894 - loss: 0.5349 - val_accuracy: 0.8884 - val_loss: 0.2846
Epoch 2/10
1314/1314 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8927 - loss: 0.2708 - val_accuracy: 0.8996 - val_loss: 0.2504
Epoch 3/10
1314/1314 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9131 - loss: 0.2293 - val_accuracy: 0.9063 - val_loss: 0.2343
Epoch 4/10
1314/1314 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9248 - loss: 0.2001 - val_accuracy: 0.9095 - val_loss: 0.2310
Epoch 5/10
1314/1314 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9335 - loss: 0.1803 - val_accuracy: 0.9071 - val_loss: 0.2373
Epoch 6/10
1314/1314 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9423 - loss: 0.1642 - val_accuracy: 0.9167 - val_loss: 0.2137
Epoch 7/10
1314/1314 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9499 - loss: 0.1484 - val_accuracy: 0.9153 - val_loss: 0.2206
Epoch 8/10
1314/1314 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9556 - loss: 0.1325 - 

In [ ]:
y_predict=model.predict(np.array(x_test))
y_pred=np.where(y_predict>0.5,1,0)

411/411 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


In [ ]:
classificationreport=classification_report(y_test,y_pred)

In [ ]:
print(classificationreport)

              precision    recall  f1-score   support

           0       0.92      0.91      0.91      6403
           1       0.91      0.92      0.92      6733

    accuracy                           0.91     13136
   macro avg       0.91      0.91      0.91     13136
weighted avg       0.91      0.91      0.91     13136



In [ ]:
confusion_matrix(y_test,y_pred)

array([[5801,  602],
       [ 527, 6206]])

In [ ]:
def predictnews(news:string)-> bool:
  text=embedding.embed_documents([news])
  prediction=model.predict(np.array(text))

  return prediction

In [ ]:
while True:
  inp=input("enter news")
  if inp=='0':
    break
  print(predictnews(inp))


In [ ]:
model.save('unreliable+news+detection+model.keras')